In [1]:
# Version 2 + Bug fix - thanks to @chinhuic

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input/NVIDIA-apex-39e153a"))
#print(os.listdir("../input/glove-global-vectors-for-word-representation"))
#print(os.listdir("../input/jigsaw-unintended-bias-in-toxicity-classification"))
#print(os.listdir("../input/fasttext-crawl-300d-2m"))

# Any results you write to the current directory are saved as output.

In [2]:
from platform import python_version
print(python_version())

3.6.6


In [3]:
# Installing Nvidia Apex
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/NVIDIA-apex-39e153a

/home/hyang/anaconda3/lib/python3.6/site-packages/pip/_internal/commands/install.py:207: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-i32jxlo_
Created temporary directory: /tmp/pip-req-tracker-h4n7awkm
Created requirements tracker '/tmp/pip-req-tracker-h4n7awkm'
Created temporary directory: /tmp/pip-install-68d_awoz
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Processing /ssd1/kudumi/input/NVIDIA-apex-39e153a
  Created temporary directory: /tmp/pip-req-build-l1f_7ila
  Added file:///ssd1/kudumi/input/NVIDIA-apex-39e153a to build tracker '/tmp/pip-req-tracker-h4n7awkm'
    Running setup.py (path:/tmp/pip-req-build-l1f_7ila/setup.py) egg_info for package from file:///ssd1/kudumi/input/NVIDIA-apex-39e153a
    Running command python setup.py egg_info
    torch.__version__  =  1.1.0
    running egg_info

In [3]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
#from apex import amp
import shutil
#from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [4]:
device=torch.device('cuda:0')

In [5]:
MAX_SEQUENCE_LENGTH = 300#220
SEED = 1234
EPOCHS = 1
Data_dir="./"
Input_dir = "../input"
WORK_DIR = "../working/"
#num_to_load=1000000                         #Train size to match time limit
#valid_size= 100000                          #Validation Size
TOXICITY_COLUMN = 'target'

In [6]:
package_dir_a = "../input/pytorch-pretrained-BERT"
sys.path.insert(0, package_dir_a)

from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam

/home/hyang/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:939: DeprecationWarning: builtin type EagerTensor has no __module__ attribute
  EagerTensor = c_api.TFE_Py_InitEagerTensor(_EagerTensorBase)
/home/hyang/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/hyang/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/_impl/keras/backend.py:4639: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/hyang/.keras/keras.json' mode='r' encoding='UTF-8'>
  _config = {}
/home/hyang/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/hyang/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning

In [7]:
BERT_MODEL_PATH = '../input/uncased_L-12_H-768_A-12/'#区分大小写
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /ssd1/kudumi/input/uncased_L-12_H-768_A-12/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/gamma with shape [768]
Loading TF

Loading TF weight bert/encoder/layer_5/output/LayerNorm/gamma with shape [768]
Loading TF weight bert/encoder/layer_5/output/dense/bias with shape [768]
Loading TF weight bert/encoder/layer_5/output/dense/kernel with shape [3072, 768]
Loading TF weight bert/encoder/layer_6/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_6/attention/output/LayerNorm/gamma with shape [768]
Loading TF weight bert/encoder/layer_6/attention/output/dense/bias with shape [768]
Loading TF weight bert/encoder/layer_6/attention/output/dense/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_6/attention/self/key/bias with shape [768]
Loading TF weight bert/encoder/layer_6/attention/self/key/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_6/attention/self/query/bias with shape [768]
Loading TF weight bert/encoder/layer_6/attention/self/query/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_6/attention/self/value/bias with shape

'../working/bert_config.json'

In [8]:
os.listdir("../working")

['pytorch_model.bin', 'bert_config.json']

In [9]:
# This is the Bert configuration file
from pytorch_pretrained_bert import BertConfig

bert_config = BertConfig('../input/uncased_L-12_H-768_A-12/'+'bert_config.json')#区分大小写

In [ ]:
#运行到这里

In [10]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(example, max_seq_length,tokenizer):
    length_stats = []
    max_seq_length -=1
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        length_stats.append( len(tokens_a)  )
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        #CLS:101;SEP:102
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a)+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens),length_stats

In [ ]:
#one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))

In [11]:
#bucket pad
def add_mask(seqs):
    tmp = seqs.clone()
    tmp_len = seqs.shape[1]
    tmp_count = tmp.shape[0]
    for i in range(tmp_count):
        flag_tmp = 1
        for j in range(tmp_len):
            if  tmp[i,j] == 0:
                tmp[i,j] = 102
                flag_tmp = 0
                break
        if flag_tmp:
            tmp[i,tmp_len-1]=102
    return tmp

class SequenceBucketCollator():
    def __init__(self, choose_length, sequence_index, length_index, label_index=None):
        self.choose_length = choose_length
        self.sequence_index = sequence_index
        self.length_index = length_index
        self.label_index = label_index
        
    def __call__(self, batch):
        batch = [torch.stack(x) for x in list(zip(*batch))]
        
        sequences = batch[self.sequence_index]
        
        
        lengths_tmp = batch[self.length_index]
        
        length = self.choose_length(lengths_tmp)
        
        mask = torch.arange(start=0, end=MAX_SEQUENCE_LENGTH, step=1) < length+1
        padded_sequences = sequences[:, mask]
        padded_sequences = add_mask(padded_sequences)
        batch[self.sequence_index] = padded_sequences
        
        if self.label_index is not None:
            return [x for i, x in enumerate(batch) if i != self.label_index], batch[self.label_index]
    
        return batch

In [12]:
BERT_MODEL_PATH = '../input/uncased_L-12_H-768_A-12/'

In [13]:
#分割数据集
train_all = pd.read_csv('./train.csv')

def get_folds(X,Y,n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits,shuffle=True, random_state= 2019)
    #skf.get_n_splits(X,y)
    fold_ids = []
    for train_index,test_index in skf.split(X,Y):
        fold_ids.append([train_index,test_index])
    return(fold_ids)

train_all['target_trans'] = np.where(train_all['target'] >= 0.5, 1, 0)

from sklearn.model_selection import KFold,StratifiedKFold
fold_ids = get_folds(train_all.drop(['target_trans'],axis=1),train_all.loc[:,'target_trans'],n_splits=5)
train_all.drop(['target_trans'],axis=1,inplace = True)
train = train_all.iloc[fold_ids[0][0]].reset_index(drop=True)
test = train_all.iloc[fold_ids[0][1]].reset_index(drop=True)

In [14]:
train.shape

(1443899, 45)

In [15]:
symbols_to_isolate = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
symbols_to_delete = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'

In [16]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
small_caps_mapping = { 
    "ᴀ": "a", "ʙ": "b", "ᴄ": "c", "ᴅ": "d", "ᴇ": "e", "ғ": "f", "ɢ": "g", "ʜ": "h", "ɪ": "i", 
    "ᴊ": "j", "ᴋ": "k", "ʟ": "l", "ᴍ": "m", "ɴ": "n", "ᴏ": "o", "ᴘ": "p", "ǫ": "q", "ʀ": "r", 
    "s": "s", "ᴛ": "t", "ᴜ": "u", "ᴠ": "v", "ᴡ": "w", "x": "x", "ʏ": "y", "ᴢ": "z"}
contraction_mapping = {
    "ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
    "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
    "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  
    "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": 
    "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", 
    "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", 
    "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", 
    "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", 
    "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", 
    "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's":"this is","that'd": "that would", 
    "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", 
    "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
    "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", 
    "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", 
    "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", 
    "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
    "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", 
    "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have",
    "trump's": "trump is", "obama's": "obama is", "canada's": "canada is", "today's": "today is"}
specail_signs = { "…": "...", "₂": "2"}
specials = ["’", "‘", "´", "`"]

In [17]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()


isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}


def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x

def handle_contractions(x):
    x = tokenizer.tokenize(x)
    return x

def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def preprocess(x):
    x = handle_punctuation(x)
    x = handle_contractions(x)
    x = fix_quote(x)
    return x

In [18]:
tqdm.pandas()

In [19]:
x_train = train['comment_text'].progress_apply(lambda x:preprocess(x))
y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]
x_test = test['comment_text'].progress_apply(lambda x:preprocess(x))

identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

# Overall
weights = np.ones((len(x_train),)) / 4

# Subgroup
weights += (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4

# Background Positive, Subgroup Negative
weights += (( (train['target'].values>=0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4

# Background Negative, Subgroup Positive
weights += (( (train['target'].values<0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
loss_weight = 1.0 / weights.mean()

y_train = np.vstack([(train['target'].values>=0.5).astype(np.int),weights]).T

max_features = 410047

100%|██████████| 360975/360975 [01:53<00:00, 3177.83it/s]


In [20]:
train['comment_text'] = x_train




In [21]:
test['comment_text'] = x_test

In [22]:
%%time
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)#区分大小写不
train_df = train
print('loaded %d records' % len(train_df))

# Make sure all comment_text values are strings
#train_df['comment_text'] = train_df['comment_text'].astype(str) 



loaded 1443899 records
CPU times: user 27.1 ms, sys: 2.02 ms, total: 29.1 ms
Wall time: 27.9 ms


In [23]:
x_train[0]

'This is so cool . It s like , would you want your mother to read this ? ?  Really great idea , well done !'

In [24]:
sequences,lengths = convert_lines(train_df["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
lengths = torch.from_numpy(np.array(   lengths     ))

train_df=train_df.fillna(0)
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target']

train_df = train_df.drop(['comment_text'],axis=1)
# convert target to 0,1
train_df['target']=(train_df['target']>=0.5).astype(float)

HBox(children=(IntProgress(value=0, max=1443899), HTML(value='')))


242


In [25]:
y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]
y_train = np.vstack([(train['target'].values>=0.5).astype(np.int),weights]).T
#y_train_torch = torch.tensor(np.hstack([y_train, y_aux_train]), dtype=torch.float32)
y_train =  np.hstack([y_train, y_aux_train])
y_train.shape

(1443899, 8)

In [26]:
test_df = test
sequences_test,lengths_test = convert_lines(test_df["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
lengths_test = torch.from_numpy(np.array(   lengths_test     ))

test_df=test_df.fillna(0)
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target']

test_df = test_df.drop(['comment_text'],axis=1)
# convert target to 0,1
test_df['target']=(test_df['target']>=0.5).astype(float)

HBox(children=(IntProgress(value=0, max=360975), HTML(value='')))


53


In [28]:
y_aux_test = test[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]
y_test = np.vstack([(test['target'].values>=0.5).astype(np.int),np.ones(test.shape[0])]).T
#y_train_torch = torch.tensor(np.hstack([y_train, y_aux_train]), dtype=torch.float32)
y_test =  np.hstack([y_test, y_aux_test])
y_test.shape

(360975, 8)

(1100000, 8)

In [29]:
X = sequences               
#y = train_df[y_columns].values[:num_to_load]
y = y_train
train_length = lengths


X_val = sequences_test                
#y_val = train_df[y_columns].values[num_to_load:]
y_val = y_test
val_length = lengths_test

In [30]:
device

device(type='cuda', index=0)

In [31]:
from torch.nn import DataParallel


In [29]:
import gc
#del model
gc.collect()

0

In [60]:
#获取显存中的tensor
def get_tensor():
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                tensor = obj
            else:
                continue
            if tensor.is_cuda:
                yield tensor
        except Exception as e:
            print(f'a exception occured : {e}')

In [61]:
#释放显存
torch.cuda.empty_cache()

In [33]:
def custom_loss(data, targets):
    ''' Define custom loss function for weighted BCE on 'target' column '''
    bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1])
    bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,1:],targets[:,2:])
    return (bce_loss_1 * loss_weight) + bce_loss_2

In [34]:
#bucket pad
#lambda lengths: np.percentile(lengths, q=95)
batch_size = 64
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), train_length,torch.tensor(y,dtype=torch.float))
train_collator = SequenceBucketCollator(lambda lengths: np.percentile(lengths, q=95), 
                                        sequence_index=0, 
                                        length_index=1, 
                                        label_index=2)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_collator)
#test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_collator)


In [33]:
#待测试


In [42]:
avg_loss = 0.
avg_accuracy = 0.
lossf=None
tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
optimizer.zero_grad()   # Bug fix - thanks to @chinhuic

HBox(children=(IntProgress(value=0, max=15625), HTML(value='')))

In [45]:
i,(x_batch, y_batch) = next(iter(   tk0 ))

In [49]:
x_batch[0][1]

tensor([ 101, 6187, 1050, 1005, 1056, 5674, 1037, 3663, 2073, 1996, 9131, 2832,
        2052, 2025, 3073, 8208, 2043, 5621, 2734, 1998, 3569, 2011, 1996, 2111,
        1012,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

In [46]:
x_batch[0]

[tensor([[ 101, 3599, 1012,  ...,    0,    0,    0],
         [ 101, 6187, 1050,  ...,    0,    0,    0],
         [ 101, 2023, 2028,  ...,    0,    0,    0],
         ...,
         [ 101, 1996, 3291,  ...,    0,    0,    0],
         [ 101, 2672, 2065,  ...,    0,    0,    0],
         [ 101, 1996, 6884,  ...,    0,    0,    0]]),
 tensor([147,  24,  26,  39, 203,  19, 166,  10,  56,  15, 218,  38,  50,  28,
          53,  34,   9,  44,  20,   9, 141,  79,  15,  49, 195,   9,   7,  29,
          23, 194,  66, 125, 151,  55,   9,  79, 101,  23, 232,  16,  26,  28,
          23,  47,  79,  31, 104,  26, 151, 233,  27, 181, 151,  18,  49,  33,
          48,   4, 150, 126, 126,  22,  47,  27])]

In [50]:
y_pred = model(x_batch[0].to(device), attention_mask=(x_batch[0]>0).to(device), labels=None)

In [60]:
del  y_batch
gc.collect()

0

In [35]:
from pytorch_pretrained_bert import OpenAIAdam

In [39]:
1+1

2

In [ ]:
output_model_file = "bert_pytorch.bin"

lr=6e-6
#batch_size = 80
accumulation_steps=2
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#model = BertForSequenceClassification.from_pretrained("../working",cache_dir=None,num_labels= 7 )
model.zero_grad()
#model = nn.DataParallel(model)
#model = model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
train = train_dataset

num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = OpenAIAdam(optimizer_grouped_parameters,
                     lr=lr,
                     warmup=0.05,
                     t_total=num_train_optimization_steps)

#model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model=model.train()

tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    #train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    optimizer.zero_grad()   # Bug fix - thanks to @chinhuic
    for i,(x_batch, y_batch) in tk0:
        #print(x_batch)
#        optimizer.zero_grad()
        y_pred = model(x_batch[0].to(device), attention_mask=(x_batch[0]>0).to(device), labels=None)
        #loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        loss = custom_loss(y_pred,y_batch.to(device))
        
        #with amp.scale_loss(loss, optimizer) as scaled_loss:
        #    scaled_loss.backward()
        loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)


#torch.save(model.state_dict(), output_model_file)

HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, max=22561), HTML(value='')))

In [61]:
import gc
del x_batch
del y_batch
#del attention_mask
gc.collect()

5947

In [62]:
del model1

NameError: name 'model1' is not defined

In [63]:
gc.collect()

3506

In [64]:
#释放显存
torch.cuda.empty_cache()

In [ ]:
torch.save(model.state_dict(), 'bert_pytorch_bucket_v8.bin')

In [ ]:
torch.save(model, 'bert_model_bucket_v8')

In [59]:
model = torch.load( 'bert_model_bucket_v8')

In [ ]:
4:30

In [35]:
tk0 = iter(valid_loader)
x_batch = next(tk0)

In [36]:
x_batch

[tensor([[ 101, 2241, 2006,  ...,    0,    0,    0],
         [ 101, 1045, 2499,  ...,    0,    0,    0],
         [ 101, 2092, 2016,  ...,    0,    0,    0],
         ...,
         [ 101, 2129, 2200,  ...,    0,    0,    0],
         [ 101, 3308, 2153,  ...,    0,    0,    0],
         [ 101, 9045, 2272,  ...,    0,    0,    0]]),
 tensor([121,  27, 107,  53,  13,  70, 186,  10,   8,  39, 210,  79,  81,  37,
         125,  88, 215,  24, 118,  37,  15, 107,  84,  45, 228,  29,  14,   8,
          10,  22,  23,  13,   7,  16,  44,  40,  20,  86,  69,  48,  26,  11,
          36, 118,  85,  29,  16,   6,  52,   9,  27,  64,  47,   6,  33,  62,
          40,  14,  88,   8, 112,  61,  90,  21])]

In [38]:
x_batch[0][0]

tensor([  101,  2241,  2006,  1996,  7928,  2061,  2521,  1010,  2009,  3544,
         1037,  2261,  1997,  1996,  8398, 14322,  4025,  2000,  2031,  4771,
         1996,  2391,  1997, 14251,  1055,  5930,  1012,  1045,  2222,  3046,
         2000,  2393,  1012,  2045,  2024,  2111,  2040,  2024,  2074,  5810,
         2205,  2524,  3753,  2000,  4553,  2505,  2367,  2030,  2047,  1012,
         2045,  2024,  2111,  2040,  2024,  2330,  2000,  2047,  2030,  2367,
         4784,  1010,  8578,  1998,  3431,  1012,  2045,  2024,  2111,  2040,
         2079,  2025,  3305,  3937,  2671,  3568,  9772,  2151,  3350,  2008,
         9755,  2007,  2037,  3653,  8663,  3401,  3512,  2094, 21951,  1012,
         1045,  2222,  2681,  2009,  2000,  2017,  8398,  4599,  2000,  5630,
         2073,  2017,  4906,  1999,  1996,  8674,  1997,  1996,  2104,  6727,
         2349,  2000,  2115,  7082,  4216,  2066,  4419,  2030, 18558,  9028,
         2015,  1012,   102,     0,     0,     0,     0,     0, 

In [40]:
pred = model1(x_batch[0].cuda(), attention_mask=(x_batch[0]>0).cuda(), labels=None)

In [41]:
x_batch[0].shape

torch.Size([64, 220])

In [47]:
# Run validation
# The following 2 lines are not needed but show how to download the model for prediction
model1.to(device)
for param in model1.parameters():
    param.requires_grad=False
model1.eval()
valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long),val_length)
test_collator = SequenceBucketCollator(lambda lengths: np.percentile(lengths, q=95), sequence_index=0, length_index=1)


valid_loader = torch.utils.data.DataLoader(valid, batch_size=64, shuffle=False, collate_fn=test_collator)

tk0 = tqdm_notebook(valid_loader)
for i,x_batch  in enumerate(tk0):
    pred = model1(x_batch[0].to(device), attention_mask=(x_batch[0]>0).to(device), labels=None)
    valid_preds[i*64:(i+1)*64]=pred[:,0].detach().cpu().squeeze().numpy()

DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.1)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): BertLayerNorm()
                (dro

DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.1)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): BertLayerNorm()
                (dro

HBox(children=(IntProgress(value=0, max=5641), HTML(value='')))

In [67]:
tk0 = tqdm_notebook(valid_loader)
for i,x_batch in enumerate(tk0):
    pred = model1(x_batch[0].to(device), attention_mask=(x_batch[0]>0).to(device), labels=None)
    valid_preds[i*64:(i+1)*64]=pred[:,0].detach().cpu().squeeze().numpy()

HBox(children=(IntProgress(value=0, max=1563), HTML(value='')))

In [55]:
#train
tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)

HBox(children=(IntProgress(value=0, max=15625), HTML(value='')))

In [43]:
tk0 = tqdm_notebook(valid_loader)

HBox(children=(IntProgress(value=0, max=1563), HTML(value='')))

In [56]:
tmp = iter(tk0)

In [63]:
tmp1[1][0][0]

tensor([[ 101, 7632, 1024,  ...,    0,    0,    0],
        [ 101, 2129, 2038,  ...,    0,    0,    0],
        [ 101, 2003, 2045,  ...,    0,    0,    0],
        ...,
        [ 101, 1036, 1036,  ...,    0,    0,    0],
        [ 101, 2307, 2391,  ...,    0,    0,    0],
        [ 101, 1996, 3291,  ...,    0,    0,    0]])

In [99]:
tmp1 = next(tmp)
tmp1[1][0][0].shape

torch.Size([64, 220])

In [48]:
# From baseline kernel

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>0.5]
    return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

In [49]:
MODEL_NAME = 'model1'
test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))

,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
2,0.963268,0.887900,homosexual_gay_or_lesbian,0.847359,2048
6,0.966319,0.877983,black,0.850372,2814
7,0.962315,0.893791,white,0.850503,4842
5,0.957113,0.923654,muslim,0.883411,3954
4,0.947416,0.946115,jewish,0.900537,1495
0,0.959573,0.950884,male,0.930618,8098
3,0.943624,0.966361,christian,0.932439,7031
1,0.957163,0.955857,female,0.933303,10221
8,0.973068,0.941337,psychiatric_or_mental_illness,0.946980,838


0.9362047291603617

In [ ]:
2	0.962591	0.895607	homosexual_gay_or_lesbian	0.863673	554
6	0.965884	0.897245	black	0.877774	758
7	0.963646	0.907982	white	0.878025	1304
5	0.954549	0.929022	muslim	0.886599	1100
4	0.941530	0.948155	jewish	0.893507	427
1	0.957923	0.951437	female	0.928526	2759
3	0.939028	0.969504	christian	0.932321	1982
0	0.964675	0.949924	male	0.940103	2170
8	0.979751	0.921096	psychiatric_or_mental_illness	0.940930	229
0.9394987909692798